In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", U

In [2]:
content_df = pd.read_csv('../data/Train_Dataset.csv')
label_df = pd.read_csv('../data/Train_Dataset_Label.csv')
train_df = content_df.merge(label_df, on='id')
train_df = train_df.fillna('EMPTY')

In [3]:
test_df = pd.read_csv('../data/Test_Dataset.csv')
test_df = test_df.fillna('EMPTY')
test_title_data = pd.read_csv('./bert-base/test_title_word_vector.csv', header=None).values
test_content_data = pd.read_csv('./bert-base/test_content_word_vector.csv', header=None).values
test_tail_data = pd.read_csv('./bert-base/test_tail_word_vector.csv', header=None).values
X_test = np.concatenate((test_title_data, test_content_data, test_tail_data), axis=1)

In [4]:
y = train_df['label'].values
train_title_data = pd.read_csv('./bert-base/train_title_word_vector.csv', header=None).values
train_content_data = pd.read_csv('./bert-base/train_content_word_vector.csv', header=None).values
train_tail_data = pd.read_csv('./bert-base/train_tail_word_vector.csv', header=None).values
X = np.concatenate((train_title_data, train_content_data, train_tail_data), axis=1)

In [29]:
model = lgb.LGBMClassifier(learning_rate=0.1, n_estimators=100)

In [23]:
len(y[y == 2]) / len(y)

0.39931880108991824

In [33]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

kf = KFold(5, shuffle=True, random_state=2019)
model_list = []
score_list = []
for index, (train_index, valid_index) in enumerate(kf.split(X, y)):
    X_train, X_valid, y_train, y_valid = X[train_index], X[valid_index], y[train_index], y[valid_index]
#     model = SVC(gamma='scale', decision_function_shape='ovo')
    model = LogisticRegression(solver='lbfgs', multi_class='multinomial', class_weight='balanced')
#     model = RandomForestClassifier(n_estimators=100, class_weight='balanced')
#     model =  MLPClassifier(batch_size=64)
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X_valid)
    valid_score = f1_score(y_valid, y_pred, average='macro')
    model_list.append(model)
    score_list.append(valid_score)
print(np.mean(score_list))

0.7473246364319038
